In [1]:
from sqlalchemy import create_engine

# Import scrapy
import scrapy

# Import the CrawlerProcess
from scrapy.crawler import CrawlerProcess

from scrapy import Selector
import requests

import pandas as pd
import numpy as np
import datetime

import warnings
warnings.filterwarnings("ignore")

In [2]:
def scrape_bout_data():

    page_links = [
        'http://ufcstats.com/statistics/events/completed',
    'http://ufcstats.com/statistics/events/completed?page=2',
    'http://ufcstats.com/statistics/events/completed?page=3',
    'http://ufcstats.com/statistics/events/completed?page=4',
    'http://ufcstats.com/statistics/events/completed?page=5',
    'http://ufcstats.com/statistics/events/completed?page=6',
    'http://ufcstats.com/statistics/events/completed?page=7',
    'http://ufcstats.com/statistics/events/completed?page=8',
    'http://ufcstats.com/statistics/events/completed?page=9',
    'http://ufcstats.com/statistics/events/completed?page=10',
    'http://ufcstats.com/statistics/events/completed?page=11',
    'http://ufcstats.com/statistics/events/completed?page=12',
    'http://ufcstats.com/statistics/events/completed?page=13',
    'http://ufcstats.com/statistics/events/completed?page=14',
    'http://ufcstats.com/statistics/events/completed?page=15',
    'http://ufcstats.com/statistics/events/completed?page=16',
    'http://ufcstats.com/statistics/events/completed?page=17',
    'http://ufcstats.com/statistics/events/completed?page=18',
    'http://ufcstats.com/statistics/events/completed?page=19',
    'http://ufcstats.com/statistics/events/completed?page=20',
    'http://ufcstats.com/statistics/events/completed?page=21'
    ]

    # Create the Spider class
    class ufc_bout_scraper(scrapy.Spider):
        name = 'ufc_bout_scraper'

        # start_requests method
        def start_requests( self ):
            for url in page_links:
                yield scrapy.Request( url = url, callback = self.parse_front )

        def parse_front(self, response):
            links = response.xpath('//a[@class="b-link b-link_style_black"]/@href').extract()
            for url in links:
                yield response.follow(url = url, callback=self.parse_pages)

        def parse_pages(self, response):
            dict_temp['results'].extend(response.xpath('/html/body/section/div/div/table/tbody/tr/td[1]/p[1]/a/i/i/text()').extract())
            dict_temp['fighter_1'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[2]/p[1]/a/text()').extract()])
            dict_temp['fighter_2'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[2]/p[2]/a/text()').extract()])
            dict_temp['weight_class'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[7]/p/text()[1]').extract()])
            dict_temp['method'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[8]/p[1]/text()').extract()])
            dict_temp['no_rounds'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[9]/p[1]/text()').extract()])
            dict_temp['end_time'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[10]/p[1]/text()').extract()])

            no_rows = len([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[10]/p[1]/text()').extract()]) #Set the number of rows in the table to extract

            dict_temp['fighter_1_strikes'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[3]/p[1]/text()').extract()])
            dict_temp['fighter_1_td'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[4]/p[1]/text()').extract()])
            dict_temp['fighter_1_sub'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[5]/p[1]/text()').extract()])
            dict_temp['fighter_1_pass'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[6]/p[1]/text()').extract()])
            dict_temp['fighter_2_strikes'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[3]/p[2]/text()').extract()])
            dict_temp['fighter_2_td'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[4]/p[2]/text()').extract()])
            dict_temp['fighter_2_sub'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[5]/p[2]/text()').extract()])
            dict_temp['fighter_2_pass'].extend([i.strip() for i in response.xpath('/html/body/section/div/div/table/tbody/tr/td[6]/p[2]/text()').extract()])

            #Set variables that are repeated --> Fix
            date = response.xpath('/html/body/section/div/div/div[1]/ul/li[1]/text()[2]').extract()[0].strip()
            dict_temp['date'].extend([date for i in range(no_rows)])
            location = response.xpath('/html/body/section/div/div/div[1]/ul/li[2]/text()[2]').extract()[0].strip()
            dict_temp['location'].extend([location for i in range(no_rows)])
            attendance = response.xpath('/html/body/section/div/div/div[1]/ul/li[3]/text()[2]').extract()[0].strip()
            dict_temp['attendance'].extend([attendance for i in range(no_rows)])
            event_name = response.css('.b-content__title-highlight::text').extract()[0].strip()
            dict_temp['event_name'].extend([event_name for i in range(no_rows)])

    #Define the dictionary
    dict_temp = {'event_name': [],
                'attendance':[],
                 'location':[],
                 'date':[],
                'results':[],
                 'fighter_1':[],
                 'fighter_2':[],
                 'weight_class':[],
                 'method':[],
                 'no_rounds':[],
                 'end_time':[],
                 'fighter_1_strikes':[],
                 'fighter_1_td':[],
                 'fighter_1_sub':[],
                 'fighter_1_pass':[],
                 'fighter_2_strikes':[],
                 'fighter_2_td':[],
                 'fighter_2_sub':[],
                 'fighter_2_pass':[]
    }

    # Run the Spider
    process = CrawlerProcess()
    process.crawl(ufc_bout_scraper)
    process.start()

    df = pd.DataFrame(dict_temp)

    return df

In [3]:
def convert_to_float(col_name_lists,bout_df):
    for c in col_name_lists:
        bout_df[c] = bout_df[c].replace('--',np.nan).astype('float')
    return bout_df

In [4]:
def convert_to_category(col_name_lists,bout_df):
    for c in col_name_lists:
        bout_df[c] = bout_df[c].astype('category')
        
    return bout_df

In [5]:
def clean_data(df):
    df.loc[(df.attendance == ""),['attendance']] = 0
    df['attendance'] = df.attendance.replace(',','',regex=True)
    
    df['attendance'] = df['attendance'].astype('float32')
    
    #df = pd.concat([df,df['location'].str.split(pat=',',n=2,expand=True)],axis=1).rename(columns={0:'city',1:'state',2:'country'})
    #df = df.drop(columns='location')
    
    df['no_rounds'] = df['no_rounds'].astype('int32')
    
    df['date'] = pd.to_datetime(df.date)
    
    round_time = df.end_time.replace(':','',regex=True).astype('int32')
    
    round_minutes = round_time // 100
    
    round_seconds = (round_time - (round_minutes * 100))/60
    
    df['end_time'] = round_minutes + round_seconds
    
    df['total_fight_time'] = ((df.no_rounds-1)*5) + df.end_time
    
    columns_to_float = ['fighter_1_strikes','fighter_1_td','fighter_1_sub','fighter_1_pass','fighter_2_strikes','fighter_2_td','fighter_2_sub','fighter_2_pass']
    
    df = convert_to_float(columns_to_float,df)
    
    columns_to_category = ['weight_class','method']
    
    df = convert_to_category(columns_to_category, df)
    
    return df

In [6]:
bout_df = scrape_bout_data()

2020-07-26 19:41:49 [scrapy.utils.log] INFO: Scrapy 2.1.0 started (bot: scrapybot)
2020-07-26 19:41:49 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.20.0, Twisted 20.3.0, Python 3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.8, Platform Windows-10-10.0.18362-SP0
2020-07-26 19:41:49 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2020-07-26 19:41:49 [scrapy.crawler] INFO: Overridden settings:
{}
2020-07-26 19:41:49 [scrapy.extensions.telnet] INFO: Telnet Password: 86d2d4a86ebc44ce
2020-07-26 19:41:49 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2020-07-26 19:41:49 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scra

2020-07-26 19:41:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/3bb030257966b022> (referer: http://ufcstats.com/statistics/events/completed?page=10)
2020-07-26 19:41:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/a8fa0c4e95512806> (referer: http://ufcstats.com/statistics/events/completed?page=20)
2020-07-26 19:41:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/20ec0061400178ca> (referer: http://ufcstats.com/statistics/events/completed?page=20)
2020-07-26 19:41:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/cedfdf8d423d500c> (referer: http://ufcstats.com/statistics/events/completed?page=20)
2020-07-26 19:41:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/31652c9267606d54> (referer: http://ufcstats.com/statistics/events/completed?page=20)
2020-07-26 19:41:52 [scrapy.core.engine] DEBUG: Crawled (200) <GET htt

2020-07-26 19:41:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/07a18ae55dfc3cd9> (referer: http://ufcstats.com/statistics/events/completed?page=21)
2020-07-26 19:41:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/04d5718ed2661e8c> (referer: http://ufcstats.com/statistics/events/completed?page=16)
2020-07-26 19:41:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/c4b6099f0d25f75e> (referer: http://ufcstats.com/statistics/events/completed?page=16)
2020-07-26 19:41:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/29b5791e51e7e832> (referer: http://ufcstats.com/statistics/events/completed?page=16)
2020-07-26 19:41:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/cb6783c39c01d896> (referer: http://ufcstats.com/statistics/events/completed?page=16)
2020-07-26 19:41:55 [scrapy.core.engine] DEBUG: Crawled (200) <GET htt

2020-07-26 19:41:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/279093302a6f44b3> (referer: http://ufcstats.com/statistics/events/completed?page=17)
2020-07-26 19:41:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/312f47c3d2f83ffa> (referer: http://ufcstats.com/statistics/events/completed?page=17)
2020-07-26 19:41:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/a2b06ca02bca14c0> (referer: http://ufcstats.com/statistics/events/completed?page=17)
2020-07-26 19:41:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/c670aa48827d6be6> (referer: http://ufcstats.com/statistics/events/completed?page=17)
2020-07-26 19:41:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/23ab42947c1990e3> (referer: http://ufcstats.com/statistics/events/completed?page=17)
2020-07-26 19:41:57 [scrapy.core.engine] DEBUG: Crawled (200) <GET htt

2020-07-26 19:42:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/ee5df903f80c6816> (referer: http://ufcstats.com/statistics/events/completed?page=18)
2020-07-26 19:42:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/a6c2f5381d575920> (referer: http://ufcstats.com/statistics/events/completed?page=18)
2020-07-26 19:42:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/13c4313ed0f744f3> (referer: http://ufcstats.com/statistics/events/completed?page=18)
2020-07-26 19:42:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/c1684f00c626f4c0> (referer: http://ufcstats.com/statistics/events/completed?page=18)
2020-07-26 19:42:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/304fcd812f12c589> (referer: http://ufcstats.com/statistics/events/completed?page=18)
2020-07-26 19:42:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET htt

...

2020-07-26 19:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/5da4e8dc02e50ac0> (referer: http://ufcstats.com/statistics/events/completed?page=7)
2020-07-26 19:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/59aaf2730b84698a> (referer: http://ufcstats.com/statistics/events/completed?page=11)
2020-07-26 19:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/cfbccfed4e4796fe> (referer: http://ufcstats.com/statistics/events/completed?page=7)
2020-07-26 19:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/2dc7f1762dc0a7ef> (referer: http://ufcstats.com/statistics/events/completed?page=7)
2020-07-26 19:42:11 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/2b4faacc16d66898> (referer: http://ufcstats.com/statistics/events/completed?page=7)
2020-07-26 19:42:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://

2020-07-26 19:42:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/354808cf38d9d73c> (referer: http://ufcstats.com/statistics/events/completed?page=8)
2020-07-26 19:42:14 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/c6becb722706c7d8> (referer: http://ufcstats.com/statistics/events/completed?page=8)
2020-07-26 19:42:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/380e8b023290d091> (referer: http://ufcstats.com/statistics/events/completed?page=8)
2020-07-26 19:42:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/80d918336163b80c> (referer: http://ufcstats.com/statistics/events/completed?page=8)
2020-07-26 19:42:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/4b2390cfaceb91d8> (referer: http://ufcstats.com/statistics/events/completed?page=6)
2020-07-26 19:42:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://u

2020-07-26 19:42:17 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/f5990c11974d8e9c> (referer: http://ufcstats.com/statistics/events/completed?page=4)
2020-07-26 19:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/d29b5c4f22c6357d> (referer: http://ufcstats.com/statistics/events/completed?page=4)
2020-07-26 19:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/2eae41f61776c60f> (referer: http://ufcstats.com/statistics/events/completed?page=4)
2020-07-26 19:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/99bf06f20491cb54> (referer: http://ufcstats.com/statistics/events/completed?page=5)
2020-07-26 19:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/ac5f67109accb482> (referer: http://ufcstats.com/statistics/events/completed?page=4)
2020-07-26 19:42:18 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://u

2020-07-26 19:42:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/4565d435005319c0> (referer: http://ufcstats.com/statistics/events/completed)
2020-07-26 19:42:20 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/b09890ba7ce1d1e2> (referer: http://ufcstats.com/statistics/events/completed)
2020-07-26 19:42:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/3ae10ac4df3df05c> (referer: http://ufcstats.com/statistics/events/completed?page=2)
2020-07-26 19:42:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/3cf68c1d17f66af7> (referer: http://ufcstats.com/statistics/events/completed?page=2)
2020-07-26 19:42:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/df05aa15b2d66f57> (referer: http://ufcstats.com/statistics/events/completed?page=2)
2020-07-26 19:42:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/ev

2020-07-26 19:42:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/b5882371e2a3900d> (referer: http://ufcstats.com/statistics/events/completed?page=3)
2020-07-26 19:42:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/84283233ec42be5f> (referer: http://ufcstats.com/statistics/events/completed?page=3)
2020-07-26 19:42:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/a7a79b8efbceaaac> (referer: http://ufcstats.com/statistics/events/completed?page=3)
2020-07-26 19:42:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/6546af7ab545b90c> (referer: http://ufcstats.com/statistics/events/completed?page=3)
2020-07-26 19:42:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://ufcstats.com/event-details/2d5fbe2103f97053> (referer: http://ufcstats.com/statistics/events/completed?page=3)
2020-07-26 19:42:24 [scrapy.core.engine] DEBUG: Crawled (200) <GET http://u

In [7]:
clean_bout_df = clean_data(bout_df)

2020-07-26 19:42:25 [numexpr.utils] INFO: NumExpr defaulting to 8 threads.


In [8]:
con = create_engine("sqlite:///../database/ufc_data.db")

In [9]:
clean_bout_df.to_sql('bout_data',con,if_exists='replace',index=False)

In [10]:
#pd.read_sql("SELECT * FROM bout_data", con)